# Homework 3

## Question 1

Fidelity Investments offers a number of "sector" mutual funds, as shown on [this page](https://fundresearch.fidelity.com/mutual-funds/category-performance-annual-total-returns/SECTOR).

Write code to download this page and extract the following variables:
* Fund name
* Fund ticker symbol
* Fund inception date
* Investment category

Store the records in a list of tuples containing the fund name, ticker, and the year of fund inception. For example, your list should have records like this:

`[('Consumer Cyclical', 'Consumer Discretionary', 'FSCPX', 1990),
 ('Consumer Cyclical', 'Leisure', 'FDLSX', 1984),
 ('Consumer Cyclical', 'Retailing', 'FSRPX', 1985), ...`
 
Notice that extraneous words like "Fidelty Select" and "Portfolio", which are common to all the funds, have been removed from fund names. Note also that the inception year is an integer.

In [1]:
import requests, re

In [2]:
url = f'https://fundresearch.fidelity.com/mutual-funds/category-performance-annual-total-returns/SECTOR'

In [3]:
req = requests.get(url)
html = req.text

In [4]:
sector_mutual_funds = re.findall('<td>+[^\&]+</td>', html)

In [5]:
list_of_tuples = []
for i in sector_mutual_funds:
    Fund_ticker_symbol = re.findall('(\()([A-Z]+)(\))', i)
    Investment_category = re.findall('(<td>)([A-Za-z\s]+)(</td>)',i)
    Fund_inception_date = re.findall('([0-9][0-9][0-9][0-9])(</td>)',i)[0][0]
    try:
        Fund_Name = re.findall('(Fidelity Select)([A-Za-z\s]+)',i)
        Fund_Name = Fund_Name[0][1].replace('Portfolio','').strip()
    except:
        pass
    tup = (Investment_category[0][1],Fund_Name,Fund_ticker_symbol[0][1],(int)(Fund_inception_date))
    list_of_tuples.append(tup)

How many funds are there? Your answer should come from a single line of code.

In [6]:
list_of_tuples

[('Consumer Cyclical', 'Consumer Discretionary', 'FSCPX', 1990),
 ('Consumer Cyclical', 'Leisure', 'FDLSX', 1984),
 ('Consumer Cyclical', 'Retailing', 'FSRPX', 1985),
 ('Consumer Defensive', 'Consumer Staples', 'FDFAX', 1985),
 ('Equity Energy', 'Energy', 'FSENX', 1981),
 ('Equity Energy', 'Natural Gas', 'FSNGX', 1993),
 ('Equity Energy', 'Energy Service', 'FSESX', 1985),
 ('Industrials', 'Industrials', 'FCYIX', 1997),
 ('Industrials', 'Transportation', 'FSRFX', 1986),
 ('Industrials', 'Air Transportation', 'FSAIX', 1985),
 ('Industrials', 'Defense and Aerospace', 'FSDAX', 1984),
 ('Consumer Cyclical', 'Construction and Housing', 'FSHOX', 1986),
 ('Consumer Cyclical', 'Automotive', 'FSAVX', 1986),
 ('Industrials', 'Environment and Alternative Energy', 'FSLEX', 1989),
 ('Natural Resources', 'Chemicals', 'FSCHX', 1985),
 ('Technology', 'IT Services', 'FBSOX', 1998),
 ('Equity Energy', 'Natural Resources', 'FNARX', 1997),
 ('Communications', 'Wireless', 'FWRLX', 2000),
 ('Communications',

## Question 2

Now process the records in your dictionary to create a new dictionary with *sector* as the key, and a list of fund tickers as the values.

Your result should look like this:

`{'Consumer Cyclical': ['FSCPX', 'FDLSX', 'FSRPX', 'FSHOX', 'FSAVX', 'FBMPX'],
 'Consumer Defensive': ['FDFAX'], ...`

In [9]:
from collections import defaultdict
Investment_category_dict = defaultdict(list)

In [10]:
for i in list_of_tuples:
    Investment_category_dict[i[0]].append(i[2])    

In [11]:
for key in Investment_category_dict:
    print(key,Investment_category_dict[key])

Consumer Cyclical ['FSCPX', 'FDLSX', 'FSRPX', 'FSHOX', 'FSAVX', 'FBMPX']
Consumer Defensive ['FDFAX']
Equity Energy ['FSENX', 'FSNGX', 'FSESX', 'FNARX']
Industrials ['FCYIX', 'FSRFX', 'FSAIX', 'FSDAX', 'FSLEX']
Natural Resources ['FSCHX', 'FSDPX']
Technology ['FBSOX', 'FSDCX', 'FSPTX', 'FSCSX', 'FSELX', 'FDCPX']
Communications ['FWRLX', 'FSTCX']
Financial ['FSPCX', 'FIDSX', 'FSRBX', 'FSVLX', 'FSLBX']
Health ['FSPHX', 'FPHAX', 'FSMEX', 'FSHCX', 'FBIOX']
Equity Precious Metals ['FSAGX']
Utilities ['FIUIX', 'FSUTX']


How many sectors are there? Again, answer with just one line of code.

In [12]:
len(Investment_category_dict)

11

## Question 3

Print a table of all years that have at least one fund inception, sorted from oldest to newest, and a count of how many funds were started that year.

Hint: It will probabably help to start by constructing a new dictionary.

In [13]:
funds_by_year = defaultdict(list)

In [14]:
for i in list_of_tuples:
    funds_by_year[i[3]].append(i[1])    

In [15]:
year_list = sorted(list(funds_by_year.keys()))

In [16]:
table_dict = defaultdict(int)
for key in year_list:
    table_dict[key] =(len(funds_by_year[key]))

In [17]:
import pandas as pd
Year = list(table_dict.keys())
Fund_count =list(table_dict.values())
table = pd.DataFrame({'Year':Year,'Fund count':Fund_count})

In [18]:
table

,Year,Fund count
0,1981,5
1,1984,2
2,1985,14
3,1986,7
4,1987,1
5,1989,1
6,1990,2
7,1993,1
8,1997,2
9,1998,2


## Question 4

Morningstar provides detailed information about mutual funds, some available for free. 

[This page](https://portfolios.morningstar.com/fund/holdings?t=FBSOX&region=usa&culture=en-US) displays holdings information for the top 25 stocks held by one of the funds.

If you look at the page source, though, you won't find the data there -- it's actually coming from another web server. The following URL returns a string of JSON-formatted text that contains the information that is being used to create the other page:

https://portfolios.morningstar.com/portfo/fund/ajax/holdings_tab?&t=FBSOX

To get started, write code to download this JSON and convert it to a dictionary called `jsn` just for this one ticker to get started.

In [9]:
jsn_url = 'https://portfolios.morningstar.com/portfo/fund/ajax/holdings_tab?&t=FBSOX'

In [10]:
req = requests.get(jsn_url)
json_string = req.text

In [11]:
import json
jsn = json.loads(json_string)

In [12]:
type(jsn)

dict

Now, `jsn` has a key `htmlStr` that contains the HTML with the information we want. Look at this HTML and the web page in your browser to come up with a regular expression you can use to extract the *ticker* and *company name* for all of the 25 holdings listed.

Now, write code that extracts this information for each of the funds. When you're done you should have a dictionary where the key is the fund ticker and the values are tuples of company ticker and name. For example, your dictionary should have a list that starts like this for the key `FBSOX`:

`[('ACN', 'Accenture PLC A'),
 ('EPAM', 'EPAM Systems Inc'),
 ('IBM', 'International Business Machines Corp'),
 ('ADBE', 'Adobe Inc'), ...`
 
Note: Be careful to check that you end up with 25 unique holdings for a fund. If you end up with duplicate matches, recall that a `set` can only contain unique values, and you can convert a `list` to a set by calling `set(my_list)`.

In [13]:
html_text = jsn['htmlStr']

In [14]:
html_text1 = html_text.replace('\n','')

In [172]:
# # ticker_list = re.findall('XNYS:IBM\n',html_text)
# ticker_list = re.findall('(:)([A-Z]+)',html_text)
# # \n">International Business Machines Corp</a></th>
# company_names = re.findall('(\\n">)([^\&]+)(</a></th>)',html_text)

In [15]:
combined_list = re.findall('(<th class="row_lbl" scope="row" align="left"><a target="_blank")(.*?)(</th>)',html_text1)

In [16]:
company_dict = {}
for i in combined_list:
    ticker = re.findall('(:)([A-Z]+)',i[1])
    company = re.findall('(>)([^\&]+)(</a>)',i[1])
    company_dict[ticker[0][1]] = company[0][1]

In [17]:
company_list =[]
for key in company_dict:
    tup = (key,company_dict[key])
    company_list.append(tup)

In [18]:
company_list

[('V', 'Visa Inc Class A'),
 ('MA', 'Mastercard Inc A'),
 ('PYPL', 'PayPal Holdings Inc'),
 ('CTSH', 'Cognizant Technology Solutions Corp A'),
 ('ACN', 'Accenture PLC A'),
 ('IBM', 'International Business Machines Corp'),
 ('WP', 'Worldpay Inc Class A'),
 ('FDC', 'First Data Corp Class A'),
 ('EPAM', 'EPAM Systems Inc'),
 ('GDDY', 'GoDaddy Inc A'),
 ('FLT', 'Fleetcor Technologies Inc'),
 ('ADP', 'Automatic Data Processing Inc'),
 ('VRTU', 'Virtusa Corp'),
 ('INTU', 'Intuit Inc'),
 ('G', 'Genpact Ltd'),
 ('ADS', 'Alliance Data Systems Corp'),
 ('GPN', 'Global Payments Inc'),
 ('ADBE', 'Adobe Inc'),
 ('BAH', 'Booz Allen Hamilton Holding Corp'),
 ('WNS', 'WNS (Holdings) Ltd ADR'),
 ('LDOS', 'Leidos Holdings Inc'),
 ('WIX', 'Wix.com Ltd'),
 ('BKI', 'Black Knight Inc A'),
 ('DXC', 'DXC Technology Co'),
 ('FISV', 'Fiserv Inc')]

In [19]:
jsn_url_base = 'https://portfolios.morningstar.com/portfo/fund/ajax/holdings_tab?&t='

In [20]:
for k in list_of_tuples:
    temp_jsn_url_base = jsn_url_base +k[2]
    req = requests.get(temp_jsn_url_base)
    json_string = req.text
    jsn = json.loads(json_string)
    html_text = jsn['htmlStr']
    html_text1 = html_text.replace('\n','')
    combined_list = re.findall('(<th class="row_lbl" scope="row" align="left"><a target="_blank")(.*?)(</th>)',html_text1)
    company_dict = {}
    for i in combined_list:
        ticker = re.findall('(:)([A-Z]+)',i[1])
        company = re.findall('(>)([^\&]+)(</a>)',i[1])
        company_dict[ticker[0][1]] = company[0][1]
        company_list =[]
    for key in company_dict:
        tup = (key,company_dict[key])
        company_list.append(tup)
    print("Top 25 Companies for:",k[2])
    print("-----------------------------------------------------------------------------------------")
    print(company_list)

Top 25 Companies for: FSCPX
-----------------------------------------------------------------------------------------
[('AMZN', 'Amazon.com Inc'), ('HD', 'The Home Depot Inc'), ('MCD', "McDonald's Corp"), ('LOW', "Lowe's Companies Inc"), ('NKE', 'Nike Inc B'), ('BKNG', 'Booking Holdings Inc'), ('TJX', 'TJX Companies Inc'), ('DLTR', 'Dollar Tree Inc'), ('RCL', 'Royal Caribbean Cruises Ltd'), ('MAR', 'Marriott International Inc Class A'), ('BURL', 'Burlington Stores Inc'), ('ROST', 'Ross Stores Inc'), ('SBUX', 'Starbucks Corp'), ('DG', 'Dollar General Corp'), ('ORLY', "O'Reilly Automotive Inc"), ('HLT', 'Hilton Worldwide Holdings Inc'), ('TSLA', 'Tesla Inc'), ('PVH', 'PVH Corp'), ('LVS', 'Las Vegas Sands Corp'), ('CCL', 'Carnival Corp'), ('AZO', 'AutoZone Inc'), ('DHI', 'D.R. Horton Inc'), ('DPZ', "Domino's Pizza Inc"), ('LEN', 'Lennar Corp'), ('ULTA', 'Ulta Beauty Inc')]


IndexError: list index out of range